[테디노트](https://teddylee777.github.io/thoughts/hyper-opt에서 해보는 HPO

HPO의 종류  
1. [기본적인 HPO](https://github.com/hyperopt/hyperopt)
2. [hyperas](https://github.com/maxpumperla/hyperas): hyperopt + keras
3. [hyperopt-sklearn](https://github.com/hyperopt/hyperopt-sklearn): hyperopt + sklearn

HPO란  
베이지안 최적화의 접근 방식을 취하는 기술

In [20]:
# 실습을 위한 boston housing dataset 로드
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 불러오기
data = load_boston()
SEED = 1

# train, test split
x_train, x_test, y_train, y_test = train_test_split(data['data'], data['target'], random_state = SEED)

print(x_train.shape, y_train.shape)

(379, 13) (379,)


In [21]:
# 평가함수 정의(RMSE 이용)

from sklearn.metrics import mean_squared_error

def RMSE(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


### HyperOpt를 활용한 XGBoost 튜닝 예제  

범위는 space를 만들어 지정해주는데 dict를 만들고 '이름'을 키 값으로 가지고 '하이퍼 파라미터 범위'를 밸류로 갖는다.  
- hp.choice(하이퍼파라미터 이름, 후보군 리스트): 후보군 리스트 중에서 하나를 선택하여 대입하면서 최적의 하이퍼 파라미터를 찾습니다.  
- hp.quniform(하이퍼 파라미터 이름, start, end, step): start, end까지 step 간격으로 생성된 후보군 중에서 최적의 하이퍼 파라미터를 찾습니다.  
- hp.uniform(하이퍼 파라미터 이름, start, end): start, end 사이의 임의의 값 중에서 최적의 하이퍼 파라미터를 찾습니다.  

더 자세한 정보는 [공식 wiki](https://github.com/hyperopt/hyperopt/wiki/FMin)에서 확인 가능

In [22]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

# regularization 후보군 정의
reg_candidate = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 5, 10, 100]

# 범위 정의. HPO의 이름을 key로 입력
space={'max_depth': hp.quniform("max_depth", 5, 15, 1),
       'learning_rate': hp.quniform ('learning_rate', 0.01, 0.05, 0.005),
       'reg_alpha' : hp.choice('reg_alpha', reg_candidate),
       'reg_lambda' : hp.choice('reg_lambda', reg_candidate),
       'subsample': hp.quniform('subsample', 0.6, 1, 0.05),
       'colsample_bytree' : hp.quniform('colsample_bytree', 0.6, 1, 0.05),
       'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
       'n_estimators': hp.quniform('n_estimators', 200, 1500, 100)
      }

# 목적 함수 정의
# n_estimators, max_depth와 같은 반드시 int 타입을 가져야 하는 hyperparamter는 int로 타입 캐스팅 합니다.
from xgboost import XGBRegressor
def hyperparameter_tuning(space):
    model=XGBRegressor(n_estimators =int(space['n_estimators']), 
                       max_depth = int(space['max_depth']), 
                       learning_rate = space['learning_rate'],
                       reg_alpha = space['reg_alpha'],
                       reg_lambda = space['reg_lambda'],
                       subsample = space['subsample'],
                       colsample_bytree = space['colsample_bytree'], 
                       min_child_weight = int(space['min_child_weight']),
                       random_state=SEED, 
                      )

    evaluation = [(x_train, y_train), (x_test, y_test)]
    
    model.fit(x_train, y_train,
              eval_set=evaluation, 
              eval_metric="rmse",
              early_stopping_rounds=20,
              verbose=0)

    pred = model.predict(x_test)
    rmse= RMSE(y_test, pred)    
    # 평가 방식 선정
    return {'loss':rmse, 'status': STATUS_OK, 'model': model}

In [23]:
# 목적 함수가 지정되었다면 fmin 함수도 최적화를 지정한다.
# fmin에는 다양한 옵션 값들을 지정할 수 있다. 지정해주는 알고리즘과 최대 반복 횟수 등을 변경해보면서 성능이 달라지는지 모니터링을 한다.

# Trials 객체 선언
trials = Trials()
# best에 최적의 하이퍼 파라미터를 return
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trials)

# 최적화된 결과를 int로 변환해야하는 파라미터는 타입 변환을 수행합니다.
best['max_depth'] = int(best['max_depth'])
best['min_child_weight'] = int(best['min_child_weight'])
best['n_estimators'] = int(best['n_estimators'])
best['reg_alpha'] = reg_candidate[int(best['reg_alpha'])]
best['reg_lambda'] = reg_candidate[int(best['reg_lambda'])]
best['random_state'] = SEED
print (best)
# 다음과 같은 결과 확인 가능
# {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 2, 
# 'n_estimators': 600, 'reg_alpha': 0.01, 'reg_lambda': 0.0001, 'subsample': 0.9, 'random_state': 1}

100%|██████████| 50/50 [00:20<00:00,  2.47trial/s, best loss: 2.7623337254527827]
{'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 600, 'reg_alpha': 0.01, 'reg_lambda': 0.0001, 'subsample': 0.9, 'random_state': 1}


In [24]:
# best에 담긴 튜닝된 하이퍼파라미터를 이용해 XGBoost 알고리즘에 적용하여 예측하자
xgb = XGBRegressor(**best)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)

print(RMSE(y_test, pred))

2.7623337254527827
